In [2]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

In [3]:
query_simtest = """
SELECT
  count(*),
  trim('"' FROM block::text )::int as block
FROM (SELECT
e->'log'->'height'::text as block
from fnode0.missing_messages
 WHERE e->'log'->'event'->4 is not null 
"""

query_order = """
 ) as s
  GROUP by s.block
 ORDER BY block ;"""


def query_run(i):
    return query_simtest + ' AND run = %s ' % i + query_order

connection = pg.connect("host='localdb' dbname=load user=load password='load'")


In [4]:
d1 = pd.read_sql_query(query_run(1),con=connection)
d2 = pd.read_sql_query(query_run(2),con=connection)
d3 = pd.read_sql_query(query_run(3),con=connection)
d4 = pd.read_sql_query(query_run(4),con=connection)
d5 = pd.read_sql_query(query_run(5),con=connection)
d6 = pd.read_sql_query(query_run(6),con=connection)

d7 = pd.read_sql_query(query_run(7),con=connection)
d8 = pd.read_sql_query(query_run(8),con=connection)
d9 = pd.read_sql_query(query_run(9),con=connection)
d10 = pd.read_sql_query(query_run(10),con=connection)
d11 = pd.read_sql_query(query_run(11),con=connection)
d12 = pd.read_sql_query(query_run(12),con=connection)


In [7]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd
from datetime import datetime


data = [
    go.Scatter(x=d1['block'], y=d1['count'], name="v5_5LFF"),
    go.Scatter(x=d2['block'], y=d2['count'], name="v5_6LFF"),
    go.Scatter(x=d3['block'], y=d3['count'], name="v5_6LF"),
    go.Scatter(x=d4['block'], y=d4['count'], name="v5_7LFF"),
    go.Scatter(x=d5['block'], y=d5['count'], name="v5_7LF_d0"),
    go.Scatter(x=d6['block'], y=d6['count'], name="v5_8LFF"),
    
    go.Scatter(x=d7['block'], y=d7['count'], name="5LFF"),
    go.Scatter(x=d8['block'], y=d8['count'], name="6LFF"),
    go.Scatter(x=d9['block'], y=d9['count'], name="6LF"),
    go.Scatter(x=d10['block'], y=d10['count'], name="7LFF"),
    go.Scatter(x=d11['block'], y=d11['count'], name="7LF"),
    go.Scatter(x=d12['block'], y=d12['count'], name="8LFF"),
]

py.iplot(data, filename = 'v6 vs v5 missing messages')